I want the model to get the best 10 matching jobs for a user. The main feature is matching the users skills to the description, qualifications, and responsibilities (these were all merged into one column in the cleaning step)
Additional features, such as the min salary and distance from the user, are handled through filtering the input data and that isn't handled here but rather in the sites source code.

In [95]:
import pandas as pd

job_df = pd.read_csv("jobs_data.csv")
job_df.set_index("job_id", inplace=True)
job_df

,employer_name,employer_logo,employer_website,employer_company_type,job_publisher,job_employment_type,job_title,job_apply_link,job_apply_is_direct,job_apply_quality_score,...,job_posting_language,job_onet_soc,job_onet_job_zone,job_occupational_categories,job_naics_code,job_naics_name,job_qualifications,job_responsibilities,job_full_description,job_salary
job_id,,,,,,,,,,,,,,,,,,,,,
j_4PDwz0Id7J8MZeAAAAAA==,Scientific Research Corporation,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,NaN,LinkedIn,Intern,Software Engineering Intern,https://www.linkedin.com/jobs/view/software-en...,False,0.6456,...,en,15113200,4,NaN,NaN,NaN,Basic knowledge of object-oriented software de...,"Works with others to design, code, implement, ...",Primary Duties & Responsibilities\n• Works wit...,26.0
QEXVBY7_62ApwQLKAAAAAA==,Accolade,https://d10j0m6hqftivr.cloudfront.net/Accolade...,http://accolade.com,NaN,ZipRecruiter,Intern,Mobile Software Engineering Intern (Android),https://www.ziprecruiter.com/c/Accolade/Job/Mo...,True,0.5677,...,en,15113200,4,"['15-1132.00: Software Developers, Applications']",NaN,NaN,Minimum Bachelor's Degree in Computer Science ...,Work with a tight-knit agile team to design an...,About Accolade\nAccolade (Nasdaq: ACCD) provid...,53.0
vM5K7KMOCEuIqV26AAAAAA==,Acima,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,NaN,LinkedIn,Fulltime,Software Development Engineer Intern,https://www.linkedin.com/jobs/view/software-de...,False,0.6224,...,en,15113200,4,NaN,NaN,NaN,Must be a rising Junior or above at an accredi...,The Software Development Engineer Intern will ...,"Upbound, Internship with Rent-A-Center Corpora...",40.0
4BVt84Fs4da_Xq1wAAAAAA==,Raytheon,https://upload.wikimedia.org/wikipedia/commons...,http://www.rtx.com,Manufacturing,ZipRecruiter,Fulltime,2024 Raytheon-Software Engineering Intern-Hybrid,https://www.ziprecruiter.com/c/Raytheon/Job/20...,False,0.5807,...,en,15113300,4,"['15-1132.00: Software Developers, Applications']",334511.0,"Search, Detection, Navigation, Guidance, Aeron...",This position can be a technical intern based ...,The duties performed in this position include ...,Date Posted:\n2024-04-24\n\nCountry:\nUnited S...,36.0
DQgrrZr27AQf8OW6AAAAAA==,Jabil Careers,NaN,NaN,NaN,Jabil Careers,Fulltime,SW development engineering intern,https://careers.jabil.com/en/jobs/job/j2385480...,False,0.7986,...,en,15113200,4,NaN,NaN,NaN,Proven proficiency across multiple disciplines...,The Software Developer is an application devel...,JOB SUMMARY\n\nThe Software Developer is an ap...,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIxoaUtK-roItTARAAAAAA==,Snowflake,https://pbs.twimg.com/profile_images/167663302...,http://www.snowflake.com,Computer Services,Ladders,Intern,Software Engineer Intern (Core/Database Engine...,https://www.theladders.com/job-listing/softwar...,False,0.5300,...,en,15113200,4,NaN,541511.0,Custom Computer Programming Services,NaN,NaN,Build the future of data. Join the Snowflake t...,24.0
6dniA1uul88XRuCKAAAAAA==,AeroVironment,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,Health Care,JobzMall,Intern,Software Engineering Intern,https://www.jobzmall.com/aerovironment/job/sof...,True,0.5473,...,en,15113200,4,NaN,6215.0,Medical and Diagnostic Laboratories,NaN,You’ll have real responsibilities that impact ...,You’ll have real responsibilities that impact ...,27.0
GRriNm76ZMC7tcBdAAAAAA==,PredictionHealth,NaN,http://www.predictionhealth.com,NaN,Job Openings,Fulltime,Software Engineer/DevOps Engineer Intern,https://predictionhealth.breezy.hr/p/094f24935...,False,0.6815,...,en,15113200,4,NaN,NaN,NaN,Exposure to GCP and/or AWS Experience with Pyt...,The intern will have direct responsibilities i...,Company profile:\n\nPredictionHealth helps doc...,22.0


In [96]:
# Our example user
user = {
    "name": "Test User",
    "skills": "python,java,c++",
    "min_salary": 20,
}

In [97]:
# function to help with pre-processing coding language names
def pre_processor(text):
    text = text.lower().replace("c#", "csharp")
    text = text.replace("c++", "cplusplus")
    text = text.replace("c+", "cplusplus")
    text = text.replace("react native", "reactnative")
    text = text.replace(".net", "dotnet")
    text = text.replace("node.js", "node")
    return text

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
vec.fit(job_df["job_full_description"])

user_tfidf = vec.transform([user["skills"]])
jobs_tfidf = vec.transform(job_df["job_full_description"])


In [103]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=10, metric="cosine")
model.fit(jobs_tfidf)

distances, indices = model.kneighbors(user_tfidf)
nearest_jobs = job_df.iloc[indices[0]]
nearest_jobs


,employer_name,employer_logo,employer_website,employer_company_type,job_publisher,job_employment_type,job_title,job_apply_link,job_apply_is_direct,job_apply_quality_score,...,job_posting_language,job_onet_soc,job_onet_job_zone,job_occupational_categories,job_naics_code,job_naics_name,job_qualifications,job_responsibilities,job_full_description,job_salary
job_id,,,,,,,,,,,,,,,,,,,,,
4sMapeOCETRb5AdNAAAAAA==,L3Harris Technologies,https://upload.wikimedia.org/wikipedia/en/2/23...,http://www.l3harris.com,NaN,WayUp,Intern,Software Engineering Intern C Salt Lake City UT,https://www.wayup.com/i-j-L3Harris-Technologie...,False,0.6632,...,en,15113200,4,NaN,NaN,NaN,NaN,NaN,Description: Job Title: Software Engineering I...,22.0
JzCFlLlsFfD_VSgnAAAAAA==,Malvern Panalytical Ltd,https://albahatha.com/wp-content/uploads/2023/...,http://www.malvern.com,NaN,Jooble,Intern,Software Engineer Internship,https://jooble.org/jdp/-8040892197942922813,False,0.6360,...,en,15113200,4,NaN,NaN,NaN,Currently pursuing a Bachelor of Science in Co...,"As a Software Intern, you'll be working on hig...","The Role\n\nAs a Software Intern, you'll be wo...",26.0
gzJwanEiHmGJY87MAAAAAA==,Hewlett-Packard/HP,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,NaN,JobzMall,Fulltime,Personal Systems Software Engineer Internship,https://www.jobzmall.com/hewlett-packard-hp/jo...,True,0.5350,...,en,15113300,4,NaN,NaN,NaN,The successful candidate will have a passion f...,Develop and maintain software applications for...,We are looking for a highly motivated and tale...,19.0
EqcbMSkcBX8TCyIIAAAAAA==,Heka Aero LLC,NaN,NaN,NaN,ZipRecruiter,Intern,Software Engineering Intern,https://www.ziprecruiter.com/c/Heka-Aero-LLC/J...,True,0.6690,...,en,15113200,4,"['15-1132.00: Software Developers, Applications']",NaN,NaN,"Grow your skill set, and feel the satisfaction...",Grow efficient use of integrated development e...,Software Engineering Intern\n• **MUST BE LOCAL...,23.0
bHD7hiOQw9Xui77bAAAAAA==,Innovative Defense Technologies (IDT),NaN,NaN,NaN,Jobvite,Intern,Intern: Software Engineer / Systems Engineer |...,https://jobs.jobvite.com/idtus/job/ovltpfws,False,0.8465,...,en,15113300,4,NaN,NaN,NaN,All applicants must be able to obtain/maintain...,NaN,Background Information:\n\nInnovative Defense ...,21.0
VeC6Nf53ta4ZUOZyAAAAAA==,Intel,https://upload.wikimedia.org/wikipedia/commons...,http://www.intel.com,Manufacturing,Adzuna,Fulltime,Software Engineer - Undergraduate Intern,https://www.adzuna.com/details/4666706527,False,0.5101,...,en,15113200,4,NaN,334413.0,Semiconductor and Related Device Manufacturing,Strong programming skills in One or more progr...,The Software Engineer-Undergraduate Intern wil...,Job Description\nJoin Intel Granulate as a Sof...,64.0
a1gTHdGi6_3aBWOmAAAAAA==,Intel Corporation,https://upload.wikimedia.org/wikipedia/commons...,http://www.intel.com,Manufacturing,LocalJobs.com,Intern,Software Engineer - Undergraduate Intern,https://www.localjobs.com/job/salt-lake-city-u...,False,0.5624,...,en,15113200,4,"['Software Engineer, Engineering/Architecture,...",334413.0,Semiconductor and Related Device Manufacturing,Strong programming skills in One or more progr...,The Software Engineer-Undergraduate Intern wil...,• *Job Description**\n\nJoin Intel Granulate a...,23.0
vKCOMpaj_JKirltcAAAAAA==,M4 Engineering,NaN,http://www.m4-engineering.com,NaN,CareerBuilder,Fulltime,Aerospace Software Engineer Intern,https://www.careerbuilder.com/job/J3Q0WP63T907...,False,0.5519,...,en,17201100,4,NaN,NaN,NaN,The successful candidate will be a self starte...,Work closely with development team of aerospac...,M4 Engineering is a premiere aerospace enginee...,22.0
d9tR2p-Bp5h1jF3aAAAAAA==,Affirm,https://upload.wikimedia.org/wikipedia/commons...,http://www.affirm.com,NaN,Salary.com,Intern,Software Engineering Intern - Summer 2024,https://www.salary.com/job/affirm/software-eng...,False,0.6239,...,en,15113200,4,NaN,NaN,NaN,Experience with one of the following languages...,You'll be paired with a